In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic

# File paths
files = {
    "TNG": "./data/Output-fidTNG_CAMELS/snap_032.hdf5",
    "EOS_0.05": "SimbaTNG-production/d7202e7--EOS-0p05/snap_032.hdf5",
    "EOS_0.1": "SimbaTNG-production/d7202e7--EOS-0p1/snap_032.hdf5",
    "EOS_0.3": "SimbaTNG-production/d7202e7--EOS-0p3/snap_032.hdf5",
    "EOS_0.5": "SimbaTNG-production/d7202e7--EOS-0p5/snap_032.hdf5",
    "EOS_0.7": "SimbaTNG-production/d7202e7--EOS-0p7/snap_032.hdf5",
    "SIMBA": "SimbaTNG-production/d7202e7--EOS-1p0/snap_032.hdf5"
}

colors = {
    "TNG": "tab:blue",
    "EOS_0.05": "tab:green",
    "EOS_0.1": "tab:red",
    "EOS_0.3": "tab:purple",
    "EOS_0.5": "tab:brown",
    "EOS_0.7": "tab:pink",
    "SIMBA": "tab:orange"
}

names = list(files.keys())  # TNG → EOS → SIMBA
bins = np.logspace(6, 9, 12)
bin_centers = 0.5 * (bins[1:] + bins[:-1])

# Compute binned mean Eddington fractions
binned_f_edd = {}
for name in names:
    fpath = files[name]
    data = h5py.File(fpath, "r")
    BH_Mass = data["PartType5/BH_Mass"][:] * 1e10
    BH_MdotBondi = data["PartType5/BH_MdotBondi"][:] * 1e10
    BH_MdotEdd = data["PartType5/BH_MdotEddington"][:] * 1e10
    f_edd = BH_MdotBondi / BH_MdotEdd

    stat, _, _ = binned_statistic(BH_Mass, f_edd, statistic='mean', bins=bins)
    binned_f_edd[name] = stat

tng_avg = binned_f_edd["TNG"]
simba_avg = binned_f_edd["SIMBA"]

# Plot
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, name in enumerate(names):
    fpath = files[name]
    data = h5py.File(fpath, "r")
    BH_Mass = data["PartType5/BH_Mass"][:] * 1e10
    BH_MdotBondi = data["PartType5/BH_MdotBondi"][:] * 1e10
    BH_MdotEdd = data["PartType5/BH_MdotEddington"][:] * 1e10
    f_edd = BH_MdotBondi / BH_MdotEdd

    axes[i].scatter(BH_Mass, f_edd, s=4, alpha=0.5, color=colors[name])
    axes[i].plot(bin_centers, binned_f_edd[name], color=colors[name], lw=2, label=f"{name}")

    # Overlay TNG average on EOS runs only
    if name.startswith("EOS"):
        axes[i].plot(bin_centers, tng_avg, color=colors["TNG"], lw=2, linestyle='--', label="TNG Avg")
        axes[i].plot(bin_centers, simba_avg, color=colors["SIMBA"], lw=2, linestyle='--', label="SIMBA Avg")

    axes[i].set_xscale("log")
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"BH Mass [$h^{-1}\,\mathrm{M_\odot}$]")
    axes[i].set_ylabel(r"Eddington Fraction $f_{\rm Edd}$")
    axes[i].set_title(f"{name}, N={len(BH_Mass)}")
    axes[i].legend(fontsize=8)

# Combined plot on last axis
ax_combined = axes[7]
for name in names:
    ax_combined.plot(bin_centers, binned_f_edd[name], color=colors[name], lw=2, label=name)
ax_combined.plot(bin_centers, tng_avg, color=colors["TNG"], lw=2, linestyle='--', label="TNG Avg")
ax_combined.plot(bin_centers, simba_avg, color=colors["SIMBA"], lw=2, linestyle='--', label="SIMBA Avg")

ax_combined.set_xscale("log")
ax_combined.set_yscale("log")
ax_combined.set_xlabel(r"BH Mass [$h^{-1}\,\mathrm{M_\odot}$]")
ax_combined.set_ylabel(r"Eddington Fraction $f_{\rm Edd}$")
ax_combined.set_title("Eddington Fraction - All Runs Comparison")
ax_combined.legend(fontsize=8)

plt.tight_layout()
plt.show()
